In [1]:
import pandas as pd
import json
import os

In [2]:
path = '/Users\Samuel.Malec\Preqin\ETL_assessment'
os.chdir(path)

## <font color=navy> Approach1: Loading tables from CSV
</font>

##### <font color=red>Creating dimensions:</font>

In [3]:
dim_ac = pd.read_csv('AssetClass.csv')
dim_geo = pd.read_csv('Countries.csv')
dim_firmAdr = pd.read_csv('FirmAdrs.csv')
dim_adr = pd.read_csv('Adrs.csv')

##### <font color=red>Creating a Fact table:</font>

In [4]:
fact_firms = pd.read_csv('Firms.csv')

#### Renaming columns 

In [5]:
dim_ac = dim_ac.rename(columns={'id':'asset_class_id'})
dim_adr = dim_adr.rename(columns={'id':'address_id'})
dim_geo = dim_geo.rename(columns = {'id': 'country_id'})
fact_firms = fact_firms.rename(columns = {'id': 'firm_id'})

#### Merging tables

In [7]:
df_fact = (fact_firms.merge(dim_ac,  on= 'asset_class_id' , how='left')
          .merge((dim_firmAdr.merge((dim_adr.merge(dim_geo, on = "country_id", how = "left")), on = 'address_id', how = 'left')), on = 'firm_id', how = 'left'))
df_fact

,firm_id,firm_name,asset_class_id,aum_mn,firm_currency,asset_class,address_id,line_1,line_2,country_id,country_name
0,1,Large investment Co,1,1241,EUR,Private Equity,1,123 Investment Street,Paris 1234,1,France
1,1,Large investment Co,1,1241,EUR,Private Equity,2,10 Second Rue,Paris 1234,1,France
2,2,Investment Co subsidiary,2,42,EUR,Private Debt,2,10 Second Rue,Paris 1234,1,France
3,3,Competing Alt Investments,5,140,USD,Hedge Funds,3,BBB King Street,Berlin A3B4,4,Germany


#### <font color=navy> Converting into Json
</font>

In [8]:
json_facts = (df_fact.groupby(['firm_name','asset_class','aum_mn','firm_currency'], as_index = True)
       .apply(lambda x: x[['address_id','line_1','line_2','country_name']].to_dict('records'))
       .reset_index()
       .rename(columns={0:'addresses'})
       .to_json(orient='records'))   
print(json.dumps(json.loads(json_facts), indent=4, sort_keys=False))

[
    {
        "firm_name": "Competing Alt Investments",
        "asset_class": "Hedge Funds",
        "aum_mn": 140,
        "firm_currency": "USD",
        "addresses": [
            {
                "address_id": 3,
                "line_1": "BBB King Street",
                "line_2": "Berlin A3B4",
                "country_name": "Germany"
            }
        ]
    },
    {
        "firm_name": "Investment Co subsidiary",
        "asset_class": "Private Debt",
        "aum_mn": 42,
        "firm_currency": "EUR",
        "addresses": [
            {
                "address_id": 2,
                "line_1": "10 Second Rue",
                "line_2": "Paris 1234",
                "country_name": "France"
            }
        ]
    },
    {
        "firm_name": "Large investment Co",
        "asset_class": "Private Equity",
        "aum_mn": 1241,
        "firm_currency": "EUR",
        "addresses": [
            {
                "address_id": 1,
                "line_1": "123

## <font color=navy> Approach2: Creating tables using pandas
</font>

In [9]:
Mdim_firmAdr = pd.DataFrame({'firm_id':[1,1,2,3],
                  'address_id':[1,2,2,3]}
                  )
Mdim_geo = pd.DataFrame({'country_id':[1,2,3,4],
                  'country_name':['France','Spain','Italy','Germany']}
                  )

Mdim_ac =  pd.DataFrame({'asset_class_id':[1,2,3,4,5,6],
                  'asset_class':['Private Equity','Private Debt','Natural Resources','Infrastructure','Hedge Funds','Real Estate']}
                  )

Mdim_adr =  pd.DataFrame({'address_id':[1,2,3],
                  'line_1':['123 Investment Street','10 Second Rue','BBB King Street'],
                  'line_2':['Paris 1234','Paris 1234','Berlin A3B4'],
                  'country_id':[1,1,4]}
                  )

Mfact_firms =  pd.DataFrame({'firm_id':[1,2,3],
                  'firm_name':['Large investment Co','Investment Co subsidiary','Competing Alt Investments'],
                  'asset_class_id':[1,2,5],
                  'aum_mn':[1241,42,140],
                  'firm_currency':['EUR','EUR','USD']}
                  )

In [10]:
mdf_fact = (Mfact_firms.merge(Mdim_ac,  on= 'asset_class_id' , how='left')
              .merge((Mdim_firmAdr.merge((Mdim_adr.merge(Mdim_geo, on = "country_id", how = "left")), on = 'address_id', how = 'left')), on = 'firm_id', how = 'left'))
mdf_fact

,firm_id,firm_name,asset_class_id,aum_mn,firm_currency,asset_class,address_id,line_1,line_2,country_id,country_name
0,1,Large investment Co,1,1241,EUR,Private Equity,1,123 Investment Street,Paris 1234,1,France
1,1,Large investment Co,1,1241,EUR,Private Equity,2,10 Second Rue,Paris 1234,1,France
2,2,Investment Co subsidiary,2,42,EUR,Private Debt,2,10 Second Rue,Paris 1234,1,France
3,3,Competing Alt Investments,5,140,USD,Hedge Funds,3,BBB King Street,Berlin A3B4,4,Germany


In [11]:
Mjson_facts = (mdf_fact.groupby(['firm_name','asset_class','aum_mn','firm_currency'], as_index = True)
       .apply(lambda x: x[['address_id','line_1','line_2','country_name']].to_dict('records'))
       .reset_index()
       .rename(columns={0:'addresses'})
       .to_json(orient='records'))   
print(json.dumps(json.loads(Mjson_facts), indent=4, sort_keys=False))

[
    {
        "firm_name": "Competing Alt Investments",
        "asset_class": "Hedge Funds",
        "aum_mn": 140,
        "firm_currency": "USD",
        "addresses": [
            {
                "address_id": 3,
                "line_1": "BBB King Street",
                "line_2": "Berlin A3B4",
                "country_name": "Germany"
            }
        ]
    },
    {
        "firm_name": "Investment Co subsidiary",
        "asset_class": "Private Debt",
        "aum_mn": 42,
        "firm_currency": "EUR",
        "addresses": [
            {
                "address_id": 2,
                "line_1": "10 Second Rue",
                "line_2": "Paris 1234",
                "country_name": "France"
            }
        ]
    },
    {
        "firm_name": "Large investment Co",
        "asset_class": "Private Equity",
        "aum_mn": 1241,
        "firm_currency": "EUR",
        "addresses": [
            {
                "address_id": 1,
                "line_1": "123